In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# sns.set_style("whitegrid")

In [ ]:
import numpy as np
import pandas as pd
import json
from scipy.special import erf
from scipy.signal import peak_prominences
from findpeaks import findpeaks
import oscfar as ocf

In [ ]:
ocf.__version__

In [ ]:
# Get the folders:
import os

results_folders = [f for f in os.listdir() if os.path.isdir(f) and 'results_' in f]

In [ ]:
json_files = [f for f in os.listdir() if '.json' in f and 'temp' not in f]
for file in json_files:
    data = json.load(open(file))
    print(data['eid'])

In [ ]:
writers = []
final_peaks = []
titles = []
models = []
convert = []
models_dict = {}

class Model:
    def __init__(self, model):
        self.model = model
for r in results_folders:
    # try:
    data = json.load(open(f"{r}/temp.json"))
    time = data['times']
    
    print(r,data['eid'])
    eid = data['eid']
    titles.append(data['eid'])
    sample = np.loadtxt(f"{r}/posterior_sample_{data['eid']}.txt")
    model = sample[:, -len(time):].mean(0)

    models.append(Model(model))
    models_dict[eid] = model
    fp = findpeaks(method='topology', lookahead=1)
    results = fp.fit(model)
    fp.plot1d()
    
    df = results['df']
    peaks = df[df['peak']][~df['valley']]#[df['score']>=0.01].sort_values(by='rank')
    prominences = np.zeros(len(df))
    peak_prominence_values, _, _ = peak_prominences(model, peaks['x'].tolist())
    for i, x in enumerate(peaks['x'].tolist()):
        prominences[x] = peak_prominence_values[i]
    

    df.insert(2, 'prominence', prominences)
    filtered = df[df['peak']][~df['valley']][df['prominence'] > 0.1]
    print(filtered)

    fig = plt.figure()
    plt.plot(time,data['profile'])
    
    plt.plot(time,model)
    plt.title(f"Time profile for event {data['eid']}")
    plt.xlabel("Time (s)")
    plt.ylabel("SNR")
    for p in filtered['x'].tolist():
        plt.axvline(time[p], c='grey', ls='--')

    fig = plt.figure()
    u, c = np.unique(sample[:,9], return_counts=True)
    plt.bar(u,c)
    plt.title(f"Components for {data['eid']}")
    plt.xlabel("Number of components")
    plt.ylabel("Counts")

    w = ocf.npz_writer(data['npz'])
    w.update_burst_parameters(
        arrival_time=(filtered['x'] * w.res_time).tolist()
    )
    w.save(f"/arc/home/clegue/auto-bb-fitburst/npz_edited/{eid}.npz")
    writers.append(w)
    final_peaks.append(ocf.peaks((filtered['x'],None)))
    convert.append(np.max(w.data_full.sum(0))/np.max(data['profile']))
    # except:
    #     continue


In [ ]:
sns.set_style("white")
ocf.utils.WaterFallImage.cmap = 'viridis'
plt.figure(figsize=(20,30))
grid = ocf.waterfall_grid(5,2)
grid.plot(writers, final_peaks, titles, 'black', adjust_t=0.1)
m = np.array(models).reshape(5,2)
r = np.array(writers).reshape(5,2)
c = np.array(convert).reshape(5,2)
for i, row in enumerate(m):
    for j, col in enumerate(row):
        grid.axes[i,j].ts.plot(r[i,j].times, col.model*c[i,j])
# plt.savefig("grid_image.jpg")

In [ ]:
import sys
sys.path.append("/arc/home/clegue/auto-bb-fitburst/grid_pdf")

In [ ]:
from grid_methods import *

In [ ]:
def plot(eid, grid, loc, model, fitburst=False):
    plot_data(eid, grid, loc, fitburst, False, True)
    dt = ocf.npz_reader(f"/arc/home/clegue/auto-bb-fitburst/npz_edited/{eid}.npz")
    plot_in_ts(grid, loc, dt.times, model, lw=4)
    # plot_grid()

In [ ]:
%matplotlib inline

ylabel = "Frequency (MHz)"
xlabel = "Time (s)"

plt.figure(figsize=(25,15))
grid = create_grid(1,3, xlabel, ylabel)
eid = 71571699
plot(eid, grid, (0,0), models_dict[eid], True)
plt.show()